
# SQL Warehousing with Databricks

Databricks SQL provides access to a complete set of features, making SQL workload easy and fully compatible with your existing legacy DW!

In this notebook, you'll find some example with more advanced SQL features, such as loops and stored procedures!

In [0]:
%run ./_resources/00-setup

## 1/ Defining variables 

Databricks SQL let you define your own variable, making it easy to parametrize your sql script.

Variable typically include schema name, or any other string to execute SQL. Here is a basic example:

In [0]:
DECLARE OR REPLACE VARIABLE myvar INT DEFAULT 17;
SELECT myvar;

## 2/ Let's execute multiple statements 

You can now use multi-statements in a script, to delete a record from fact_sales and insert a new record, all in one go:

In [0]:
CREATE TABLE IF NOT EXISTS fact_sales (
    sale_id INT,
    product_id INT,
    customer_id INT,
    sale_date DATE,
    quantity INT,
    total_amount DECIMAL(10, 2)
);

INSERT INTO fact_sales (sale_id, product_id, customer_id, sale_date, quantity, total_amount) VALUES
(1, 101, 1001, '2023-01-15', 2, 29.99),
(2, 102, 1002, '2023-01-16', 1, 19.99),
(3, 103, 1003, '2023-01-17', 5, 99.95),
(4, 101, 1004, '2023-01-18', 3, 44.97),
(5, 104, 1005, '2023-01-19', 4, 79.96);

In [0]:
-- Add and delete records from fact tables multi-statement example 

BEGIN
  DELETE FROM fact_sales WHERE sale_id = 1;
  INSERT INTO fact_sales (sale_id, product_id, customer_id, sale_date, quantity, total_amount) VALUES (6, 105, 1006, '2023-01-20', 2, 59.98);
  SELECT * FROM fact_sales;
END;

## 2/ Stored procedures with DBSQL

In this example, we will create a stored procedure to calculate our revenue. This is a basic example, much more complete procedure can be created:

*Disclaimer: Stored procedures might not be available yet in your workspace - ask your account team - make sure you use the latest DBR version*

In [0]:
-- Stored procedures run only on specific DBR versions and may need to be enabled  
CREATE OR REPLACE PROCEDURE total_revenue(IN price_sold DOUBLE, IN units_sold INT, OUT revenue DOUBLE)
LANGUAGE SQL
SQL SECURITY INVOKER
AS BEGIN
  SET revenue = price_sold * units_sold;
END;

In [0]:
DECLARE revenue DOUBLE DEFAULT 0;
CALL total_revenue(100.99, 3, revenue);
SELECT revenue;

## 3/ Recursive loop example 

In [0]:
WITH RECURSIVE revenue_facts AS (
  SELECT 0 AS price_sold, 0 AS units_sold, 0 AS revenue
  UNION ALL
  SELECT  
    CASE WHEN r.quantity != 0 THEN r.total_amount / r.quantity ELSE 0 END AS price_sold,
    r.quantity AS units_sold,
    r.total_amount AS revenue
  FROM fact_sales AS r
)

SELECT * FROM revenue_facts;

## 4/ For loop example 

In [0]:
-- For loop statement 
-- sum all units_sold from a given table
BEGIN
  DECLARE total_units_sold INT DEFAULT 0;
  sumNumbers: FOR row AS SELECT quantity FROM fact_sales DO
    SET total_units_sold = total_units_sold + row.quantity;
  END FOR sumNumbers;
  VALUES (total_units_sold);
END;

## 5/ While loop example 

In [0]:

-- sum up all odd numbers from 1 through 10
  BEGIN
    DECLARE sum INT DEFAULT 0;
    DECLARE num INT DEFAULT 0;
    sumNumbers: WHILE num < 10 DO
      SET num = num + 1;
      IF num % 2 = 0 THEN
        ITERATE sumNumbers;
      END IF;
      SET sum = sum + num;
    END WHILE sumNumbers;
    VALUES (sum);
  END;


That's it, you're now ready to migrate your existing SQL script to Databricks!